In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import dill
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, roc_auc_score, recall_score, f1_score

import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

path = "../runner/results/"

import pathlib

path = pathlib.Path(path)


In [4]:
import sys
sys.path.append("../runner/")

import experiment_pipeline
#import paper_output

toy_set_params = experiment_pipeline.toy_set_params
    

def get_truth(params):
    strong=params["strong"]
    weak=params["weak"]
    irrel=params["irr"]
    truth = [True] * (strong + weak) + [False] * irrel
    return truth

def get_truthAR(params):
    strong=params["strong"]
    weak=params["weak"]
    irrel=params["irr"]
    truth = [2] * strong + [1]*weak + [0] * irrel
    return truth




c:\users\mirek\appdata\local\pypoetry\cache\virtualenvs\ijcnn-exp-mobdofid-py3.7\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.regression module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\mirek\appdata\local\pypoetry\cache\virtualenvs\ijcnn-exp-mobdofid-py3.7\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:

toy_set_params

{'Set1': {'n': 150, 'strong': 6, 'weak': 0, 'irr': 6},
 'Set2': {'n': 150, 'strong': 0, 'weak': 6, 'irr': 6},
 'Set3': {'n': 150, 'strong': 3, 'weak': 4, 'irr': 3},
 'Set4': {'n': 256, 'strong': 6, 'weak': 6, 'irr': 6},
 'Set5': {'n': 512, 'strong': 1, 'weak': 2, 'irr': 11},
 'Set6': {'n': 200, 'strong': 1, 'weak': 20, 'irr': 0},
 'Set7': {'n': 200, 'strong': 1, 'weak': 20, 'irr': 20},
 'Set8': {'n': 2000, 'strong': 10, 'weak': 10, 'irr': 50}}

In [6]:
filepath = path / "paper.dat"
results = []
toy = {}

with open(filepath,"rb") as file:
    content = dill.load(file=file)

    for set_and_model, results in content.items():
        if len(results) > 0:
            toy.setdefault(set_and_model, []).append(results)

In [7]:
    ARModels = ["RF","SQ"] 

only_ARModels = {k:v for (k,v) in toy.items() if k[1] in ARModels}

In [8]:
only_ARModels

{('Set1',
  'RF'): [[{'train_scores': 1.0,
    'features': array([ True,  True,  True,  True,  True,  True, False, False, False,
           False, False, False]),
    'runtime': 0.6505916118621826,
    'setname': 'Set1',
    'modelname': 'RF'},
   {'train_scores': 1.0,
    'features': array([ True,  True,  True,  True,  True,  True, False, False, False,
           False, False, False]),
    'runtime': 0.6656055450439453,
    'setname': 'Set1',
    'modelname': 'RF'},
   {'train_scores': 1.0,
    'features': array([ True,  True,  True,  True,  True,  True, False, False, False,
           False, False, False]),
    'runtime': 0.6415832042694092,
    'setname': 'Set1',
    'modelname': 'RF'},
   {'train_scores': 1.0,
    'features': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True]),
    'runtime': 0.5935397148132324,
    'setname': 'Set1',
    'modelname': 'RF'},
   {'train_scores': 1.0,
    'features': array([ True,  True,  True,  True

In [9]:
 unpacked_lists = pd.DataFrame(only_ARModels).T.explode(0)
 feature_frame = unpacked_lists.applymap(lambda x: x["runtime"])

In [10]:
feature_frame

0
Set1 RF   0.650592
     RF   0.665606
     RF   0.641583
     RF   0.593540
     RF   0.641583
...            ...
Set8 SQ  75.973705
     SQ  82.633857
     SQ  74.136669
     SQ  76.321213
     SQ  81.912192

[160 rows x 1 columns]

In [11]:
import fsmodel
import import_data

In [12]:
toy = import_data.Toyset()

X,y = toy.get_bootstraps()[0]

X.shape


(500, 30)

In [27]:

rf = fsmodel.RF()

%time rf.fit(X,y)

INFO:root:0.912


Wall time: 7.05 s


In [24]:

sq = fsmodel.SQ()

%time sq.fit(X,y)

DEBUG:squamish.main:RF score 0.8559999999999999
DEBUG:squamish.main:importances [235 230 245 224 229   0   0   0  25  40  25  23  26  33  45  26  25  33
  43  42  24  42  24  28  18  26  23  57  21  30]


Finished loading model, total used 100 iterations


DEBUG:squamish.main:Features from Boruta: [0 1 2 3 4 5 6 7]
DEBUG:squamish.main:Features from RF: [0 1 2 3 4]
INFO:squamish.main:Relevance Classes: [2 2 2 2 2 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
INFO:root:1.0


Wall time: 10.2 s


# Precision and Recall per Relevance Type
questions:
how sensitive is each method on different types of relevance class?

In [101]:
# We only take truth if relevance type (1 or 2) 
def get_truth_onetype(params, reltype):
    truth = get_truthAR(params)
    truth = np.array(truth)
    if reltype==0:
        raise Exception("0 makes no sense")
    return truth == reltype



def rowfunc(row, reltype, scorefunc):
    ix = row.name
    data = ix[0]
    method = ix[1]

    features = row[0]

    truth = get_truth_onetype(toy_set_params[data], reltype)
    pred = features == reltype
    score  = scorefunc(truth, pred,zero_division=0)
    
    if score == 0:
        score = np.nan
    return score


def convert_frame(series, name):
    return series.to_frame().rename(columns={0:name}).groupby(level=[0,1]).mean()

In [102]:
unpacked_lists = pd.DataFrame(only_ARModels).T.explode(0)
feature_frame = unpacked_lists.applymap(lambda x: x["features"])

In [103]:
weakly_precision = convert_frame(feature_frame.apply(rowfunc, axis=1,args=[1,precision_score]), "precision")
weakly_recall = convert_frame(feature_frame.apply(rowfunc, axis=1,args=[1,recall_score]), "recall")

weakly = pd.concat([weakly_precision,weakly_recall],axis=1).unstack(level=0).T

In [104]:
weakly

FRI        SQ
precision Set1       NaN       NaN
          Set2  1.000000  0.893333
          Set3  1.000000  1.000000
          Set4  1.000000  0.981481
          Set5  1.000000  1.000000
          Set6  1.000000  1.000000
          Set7  0.995238  1.000000
          Set8  0.981818  1.000000
recall    Set1       NaN       NaN
          Set2  1.000000  1.000000
          Set3  1.000000  0.700000
          Set4  1.000000  0.703704
          Set5  1.000000  0.500000
          Set6  1.000000  0.900000
          Set7  1.000000  0.900000
          Set8  1.000000  0.890000

In [105]:
weakly.loc["precision"].mean()

FRI    0.996722
SQ     0.982116
dtype: float64

In [106]:
weakly.groupby(level=0).mean()

,FRI,SQ
precision,0.996722,0.982116
recall,1.000000,0.799101


# Strongly

In [107]:
strongly_precision = convert_frame(feature_frame.apply(rowfunc, axis=1,args=[2,precision_score]), "precision")
strongly_recall = convert_frame(feature_frame.apply(rowfunc, axis=1,args=[2,recall_score]), "recall")

strongly = pd.concat([strongly_precision,strongly_recall],axis=1).unstack(level=0).T

In [108]:
strongly

FRI        SQ
precision Set1  1.0  1.000000
          Set2  NaN       NaN
          Set3  1.0  0.750000
          Set4  1.0  0.854762
          Set5  1.0  0.500000
          Set6  1.0  0.500000
          Set7  1.0  0.483333
          Set8  1.0  0.887529
recall    Set1  1.0  0.966667
          Set2  NaN       NaN
          Set3  1.0  1.000000
          Set4  1.0  0.983333
          Set5  1.0  1.000000
          Set6  1.0  1.000000
          Set7  1.0  1.000000
          Set8  1.0  1.000000

In [109]:
strongly.groupby(level=0).mean()

,FRI,SQ
precision,1.0,0.710803
recall,1.0,0.992857


In [110]:
combined = pd.concat([weakly,strongly],axis=1,keys=["Weakly","Strongly"]).round(decimals=2)

In [111]:
combined

Weakly       Strongly      
                  FRI    SQ      FRI    SQ
precision Set1    NaN   NaN      1.0  1.00
          Set2   1.00  0.89      NaN   NaN
          Set3   1.00  1.00      1.0  0.75
          Set4   1.00  0.98      1.0  0.85
          Set5   1.00  1.00      1.0  0.50
          Set6   1.00  1.00      1.0  0.50
          Set7   1.00  1.00      1.0  0.48
          Set8   0.98  1.00      1.0  0.89
recall    Set1    NaN   NaN      1.0  0.97
          Set2   1.00  1.00      NaN   NaN
          Set3   1.00  0.70      1.0  1.00
          Set4   1.00  0.70      1.0  0.98
          Set5   1.00  0.50      1.0  1.00
          Set6   1.00  0.90      1.0  1.00
          Set7   1.00  0.90      1.0  1.00
          Set8   1.00  0.89      1.0  1.00

In [116]:
combined.replace(np.nan,"-")

Weakly       Strongly      
                  FRI    SQ      FRI    SQ
precision Set1      -     -        1     1
          Set2      1  0.89        -     -
          Set3      1     1        1  0.75
          Set4      1  0.98        1  0.85
          Set5      1     1        1   0.5
          Set6      1     1        1   0.5
          Set7      1     1        1  0.48
          Set8   0.98     1        1  0.89
recall    Set1      -     -        1  0.97
          Set2      1     1        -     -
          Set3      1   0.7        1     1
          Set4      1   0.7        1  0.98
          Set5      1   0.5        1     1
          Set6      1   0.9        1     1
          Set7      1   0.9        1     1
          Set8      1  0.89        1     1

In [114]:
import paper_output

In [115]:
paper_output.print_df_astable(combined)

'\\begin{tabular}{llrrrr}\n\\toprule\n       &      & Weakly &       & Strongly &       \\\\\n       &      &    FRI &    SQ &      FRI &    SQ \\\\\n\\midrule\n\\textbf{precision} & \\textbf{Set1} &    NaN &   NaN &      1.0 &  1.00 \\\\\n       & \\textbf{Set2} &   1.00 &  0.89 &      NaN &   NaN \\\\\n       & \\textbf{Set3} &   1.00 &  1.00 &      1.0 &  0.75 \\\\\n       & \\textbf{Set4} &   1.00 &  0.98 &      1.0 &  0.85 \\\\\n       & \\textbf{Set5} &   1.00 &  1.00 &      1.0 &  0.50 \\\\\n       & \\textbf{Set6} &   1.00 &  1.00 &      1.0 &  0.50 \\\\\n       & \\textbf{Set7} &   1.00 &  1.00 &      1.0 &  0.48 \\\\\n       & \\textbf{Set8} &   0.98 &  1.00 &      1.0 &  0.89 \\\\\n\\textbf{recall} & \\textbf{Set1} &    NaN &   NaN &      1.0 &  0.97 \\\\\n       & \\textbf{Set2} &   1.00 &  1.00 &      NaN &   NaN \\\\\n       & \\textbf{Set3} &   1.00 &  0.70 &      1.0 &  1.00 \\\\\n       & \\textbf{Set4} &   1.00 &  0.70 &      1.0 &  0.98 \\\\\n       & \\textbf{Set5} 

In [117]:
combined_mean = combined.groupby(level=0).mean()

In [118]:
paper_output.print_df_astable(combined_mean)

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &    Weakly &           & Strongly &           \\\\\n{} &       FRI &        SQ &      FRI &        SQ \\\\\n\\midrule\n\\textbf{precision} &  0.997143 &  0.981429 &      1.0 &  0.710000 \\\\\n\\textbf{recall   } &  1.000000 &  0.798571 &      1.0 &  0.992857 \\\\\n\\bottomrule\n\\end{tabular}\n'